<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **SpaceX  Falcon 9 First Stage Landing Prediction**


# Part 6: SpaceX Launches — Interactive Map & Proximity Analysis (Folium)

**Short description:**  
This notebook loads SpaceX launch geolocation data, creates an interactive Folium map with launch-site markers and clustered launch events, colors markers by success/failure, and calculates distances from a selected launch site to nearby coastline/roads/rail/city points, visualizing them with lines and distance labels.

**Context:**

The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.

**Objectives**
- Load launch geolocation data and prepare site-level summaries.
- Build an interactive Folium map showing launch sites, clustered launch events and success/failure markers.
- Add mouse-position functionality for coordinate inspection.
- Compute haversine distances between a launch site and points of interest (coastline, highway, railway, city) and visualize distances and connecting lines on the map.
- Export the map (or save) for interactive inspection and document the steps for reproducibility.

**Notice about documentation:**  
The original notebook submission (course assignment) was kept intact. I have **only modified documentation (comments, headings, markdown)** and made **minimal, necessary corrections** to ensure the notebook runs without errors. All rights related to the lab/workshop design and original exercise belong exclusively to **IBM Corporation**. This notebook includes additional documentation for clarity, but the intellectual property of the original exercise is retained by IBM.

---

## Table of contents

1. Dependencies & execution instructions  
2. Load geolocation data & prepare site-level summary  
3. Base Folium map, JSC marker and launch-site markers  
4. Clustered launch markers colored by success/failure  
5. Mouse position utility  
6. Distance calculations (haversine) and drawing lines/labels  
7. Notes & reproducibility


## 1) Dependencies & execution instructions

This section installs and imports required Python packages.  

**Recommended local execution steps:**

1. Create and activate a Python virtual environment:
   - `python -m venv venv`
   - `source venv/bin/activate` (macOS / Linux) or `venv\Scripts\activate` (Windows)
2. Install dependencies:
   - `pip install -r requirements.txt`
3. Launch Jupyter Notebook:
   - `jupyter notebook`
4. Open this notebook and run cells top-to-bottom.

**Note:** Folium maps usually render in Jupyter notebooks; if they do not display inline, save the map (site_map.save("map.html")) and open that file in a browser.

In [1]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon
from math import sin, cos, sqrt, atan2, radians

c:\Users\aleje\OneDrive - Universidad EAFIT\Escritorio\Alejandro\Academia\Portfolio\IBM Data Science Professional Certificate\10_Applied Data Science Capstone\6_Interactive Visual Analytics_Folium\venv\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


## 2) Load geolocation data & prepare site-level summary

Load `spacex_launch_geo.csv` from the provided URL, select relevant columns (`Launch Site`, `Lat`, `Long`, `class`), and create `launch_sites_df` with unique launch site coordinates for plotting site markers.


In [2]:
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df=pd.read_csv(URL)

In [3]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


## 3) Base Folium map, JSC marker and launch-site markers

Create a Folium map centered around NASA Johnson Space Center (JSC), add a labeled marker/circle for JSC, then add circles and text-label markers for each launch site (using `DivIcon`).

In [4]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [5]:
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
marker = folium.map.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [6]:
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
for element in launch_sites_df.values:
    coordinates = [element[1],element[2]]
    circle = folium.Circle(coordinates, radius=1000, color='#d35400', fill=True).add_child(folium.Popup(element[0]))
    marker = folium.map.Marker(
    coordinates,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % element[0],
        )
    )
    site_map.add_child(circle)
    site_map.add_child(marker)
site_map

## 4) Clustered launch markers colored by success/failure

Use `MarkerCluster` to add all launch events as clustered markers. Color marker icons green for success (`class==1`) and red for failure (`class==0`). Add `MousePosition` plugin to aid coordinate inspection.

In [7]:
spacex_df.head(10)

,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS LC-40,28.562302,-80.577356,0
2,CCAFS LC-40,28.562302,-80.577356,0
3,CCAFS LC-40,28.562302,-80.577356,0
4,CCAFS LC-40,28.562302,-80.577356,0
5,CCAFS LC-40,28.562302,-80.577356,0
6,CCAFS LC-40,28.562302,-80.577356,0
7,CCAFS LC-40,28.562302,-80.577356,0
8,CCAFS LC-40,28.562302,-80.577356,0
9,CCAFS LC-40,28.562302,-80.577356,0


In [8]:
marker_cluster = MarkerCluster()

# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
def assign_marker_color(outcome):
    if outcome == 1:
        return 'green'
    else:
        return 'red'

spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.head()

,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


In [9]:
site_map.add_child(marker_cluster)

for index, record in spacex_df.iterrows():
    coordinates = [record['Lat'],record['Long']]
    marker = folium.Marker(
        coordinates,
        icon=folium.Icon(
            color='white',
            icon_color = record['marker_color']
            )
        )
    marker_cluster.add_child(marker)
    

site_map

## 5) Mouse position utility

Add mouse location display (lat/lon) to the map to help inspect coordinates interactively during exploration.

In [10]:
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

## 6) Distance calculations (haversine) and drawing lines/labels

Define a haversine-based `calculate_distance` function to compute distances (km) between two coordinates. Plot distance markers, polylines between launch site and coastline/highway/rail/city points, and annotate distances using `DivIcon`.

In [11]:
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [12]:
launch_coordinate = [28.56322, -80.57681]
coastline_coordinate = [28.56334, -80.56799]
distance_coastline = calculate_distance(launch_coordinate[0], launch_coordinate[1], coastline_coordinate[0], coastline_coordinate[1])

distance_marker = folium.Marker(
    coastline_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
        )
    )
site_map.add_child(distance_marker)
site_map

In [13]:
coordinates = [launch_coordinate,coastline_coordinate]
lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

In [14]:
highway_coordinate = [28.411780, -80.820630]

lines=folium.PolyLine(locations=[launch_coordinate,highway_coordinate], weight=1)
site_map.add_child(lines)
distance_highway = calculate_distance(launch_coordinate[0], launch_coordinate[1], highway_coordinate[0], highway_coordinate[1])
distance_circle = folium.Marker(
    highway_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#252526;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway),
        )
    )
site_map.add_child(distance_circle)
site_map

In [15]:
railway_coordinate = [28.55752, -80.80155]

lines=folium.PolyLine(locations=[launch_coordinate,railway_coordinate], weight=1)
site_map.add_child(lines)
distance_railway = calculate_distance(launch_coordinate[0], launch_coordinate[1], railway_coordinate[0], railway_coordinate[1])
distance_circle = folium.Marker(
    railway_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#252526;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railway),
        )
    )
site_map.add_child(distance_circle)
site_map

In [16]:
city_coordinate = [28.61200, -80.80788]

lines=folium.PolyLine(locations=[launch_coordinate,city_coordinate], weight=1)
site_map.add_child(lines)
distance_city = calculate_distance(launch_coordinate[0], launch_coordinate[1], city_coordinate[0], city_coordinate[1])
distance_circle = folium.Marker(
    city_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#252526;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city),
        )
    )
site_map.add_child(distance_circle)
site_map

## 7) Notes & reproducibility

- Folium maps render best in Jupyter notebooks. If map does not display, use `site_map.save("site_map.html")`.
- The dataset URL is public; if offline, download `spacex_launch_geo.csv` and update the `URL` variable.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2022-01-01)


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
